<a href="https://colab.research.google.com/github/swilsonmfc/pandas/blob/main/4_Lesson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 4
* Aggregations
* Split-Apply-Combine
* Pivot & Pivot Table
* Reshape Long & Wide

# Setup

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn

# Data

In [3]:
red_df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')
red_df['wine'] = 'red'
red_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,red
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,red
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,red
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,red
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,red
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,red
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,red
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5,red


In [4]:
white_df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', sep=';')
white_df['wine'] = 'white'
white_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6,white
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6,white
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6,white
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,white
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,white
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,white
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,white
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,white
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,white


# Concat

In [5]:
wine_df = pd.concat([red_df, white_df])
wine_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,white
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,white
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,white
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,white


# Describe

In [6]:
wine_df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.215307,0.339666,0.318633,5.443235,0.056034,30.525319,115.744574,0.994697,3.218501,0.531268,10.491801,5.818378
std,1.296434,0.164636,0.145318,4.757804,0.035034,17.749400,56.521855,0.002999,0.160787,0.148806,1.192712,0.873255
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000


# Aggregations

In [9]:
# First approach we can methods on series to build aggregations
wine_df.chlorides.mean()

0.0560338617823606

In [10]:
# We can run a method across all columns
wine_df.mean()

fixed acidity             7.215307
volatile acidity          0.339666
citric acid               0.318633
residual sugar            5.443235
chlorides                 0.056034
free sulfur dioxide      30.525319
total sulfur dioxide    115.744574
density                   0.994697
pH                        3.218501
sulphates                 0.531268
alcohol                  10.491801
quality                   5.818378
dtype: float64

## Agg

In [25]:
# We can run a function to aggregate over the DataFrame
wine_df.agg(np.mean)

fixed acidity             7.215307
volatile acidity          0.339666
citric acid               0.318633
residual sugar            5.443235
chlorides                 0.056034
free sulfur dioxide      30.525319
total sulfur dioxide    115.744574
density                   0.994697
pH                        3.218501
sulphates                 0.531268
alcohol                  10.491801
quality                   5.818378
dtype: float64

In [27]:
# We can agg multiple functions
wine_df.agg([np.min, np.max, np.mean])

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine
amin,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000,red
amax,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000,white
mean,7.215307,0.339666,0.318633,5.443235,0.056034,30.525319,115.744574,0.994697,3.218501,0.531268,10.491801,5.818378,NaN


In [29]:
# We can selectively agg certain columns
wine_df.agg({'pH':[np.min, np.max], 'quality':[np.mean]})

,pH,quality
amax,4.01,NaN
amin,2.72,NaN
mean,NaN,5.818378


In [32]:
# We can pass functions to agg including anonymous / lambdas
wine_df.agg({'pH': lambda x: x.max()})

pH    4.01
dtype: float64

# Split-Apply-Combine
* Hadley Wickham 2011 paper: The Split-Apply-Combine Strategy for Data Analysis
* Wickham has written several very popular packages in R including dplyr & ggplot2

## Group by DataFrame

In [13]:
# Split the data frame by the type of wine
# Object type returned is a DataFrameGroupBy
grp = wine_df.groupby('wine')
type(grp)

pandas.core.groupby.generic.DataFrameGroupBy

In [14]:
# Look at the first five rows by applying the head function to each group
grp.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,white
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,white
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,white
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white


In [15]:
# Finally combine the results of the descibe function applied to each group
grp.describe()

fixed acidity                                ... quality                    
              count      mean       std  min  25%  ...     min  25%  50%  75%  max
wine                                               ...                            
red          1599.0  8.319637  1.741096  4.6  7.1  ...     3.0  5.0  6.0  6.0  8.0
white        4898.0  6.854788  0.843868  3.8  6.3  ...     3.0  5.0  6.0  6.0  9.0

[2 rows x 96 columns]

## Group by Series

In [18]:
# Now let's group by wine (red / white) but select a column
# Note the return type is SeriesGroupBy
grp = wine_df.groupby('wine').quality
type(grp)

pandas.core.groupby.generic.SeriesGroupBy

In [19]:
# Similar to DataFrameGroupBy, with SeriesGroupBym we can show the first 5 values 
grp.head()

0    5
1    5
2    5
3    6
4    5
0    6
1    6
2    6
3    6
4    6
Name: quality, dtype: int64

In [21]:
# Apply a mean to each group & combine into a dataframe
quality_df = grp.mean()
quality_df

wine
red      5.636023
white    5.877909
Name: quality, dtype: float64

In [33]:
# Compactly produce the average quality by wine type
# Equivalently wine_df.groupby('wine').quality.mean()
wine_df.groupby('wine')['quality'].mean()

wine
red      5.636023
white    5.877909
Name: quality, dtype: float64

## Apply vs Agg
* Combining Agg & Apply with group by is a powerful combination
* Apply will apply the function to each group
* Agg aggregates each column for each group

In [34]:
# First, let's use apply to get the average quality by wine type
wine_df.groupby('wine').quality.apply(np.mean)

wine
red      5.636023
white    5.877909
Name: quality, dtype: float64

In [35]:
# Then let's pass in a lambda
wine_df.groupby('wine').quality.apply(lambda x: x.mean())

wine
red      5.636023
white    5.877909
Name: quality, dtype: float64

In [38]:
# Note, we can design or construct our own functions
wine_df.groupby('wine').quality.apply(lambda x: np.where(x == 5, 1, 0))

wine
red      [1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, ...
white    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, ...
Name: quality, dtype: object

In [43]:
# Agg offers more flexibility over apply - we can pass multiple functions
# Equivalent : wine_df.groupby('wine').agg({'quality': ['min', 'max', 'mean']})
wine_df.groupby('wine').agg({'quality': [np.min, np.max, np.mean]})

quality              
          min max      mean
wine                       
red         3   8  5.636023
white       3   9  5.877909

In [42]:
# Agg can operate on multiple columns
wine_df.groupby('wine').agg({'quality': [np.min, np.max, np.mean], 'pH':[np.median, np.sum, np.mean]})

quality                    pH                    
         amin amax      mean median       sum      mean
wine                                                   
red         3    8  5.636023   3.31   5294.47  3.311113
white       3    9  5.877909   3.18  15616.13  3.188267

## Multi-Index

In [44]:
# Columns from agg are Multi-Index 
grp_df = wine_df.groupby('wine').agg({'quality': [np.min, np.max, np.mean], 'pH':[np.median, np.sum, np.mean]})
grp_df.columns

MultiIndex([('quality',   'amin'),
            ('quality',   'amax'),
            ('quality',   'mean'),
            (     'pH', 'median'),
            (     'pH',    'sum'),
            (     'pH',   'mean')],
           )

In [46]:
# Accessing - Selecting at level 0
grp_df['quality'] 

,amin,amax,mean
wine,,,
red,3,8,5.636023
white,3,9,5.877909


In [48]:
# Nested columns are accessed as a tuple
grp_df[('quality', 'mean')]

wine
red      5.636023
white    5.877909
Name: (quality, mean), dtype: float64

In [50]:
# MultiIndex columns are kept in layers
# Layer 0 is shown once in print, but present on every column
grp_df.columns.get_level_values(0) 

Index(['quality', 'quality', 'quality', 'pH', 'pH', 'pH'], dtype='object')

In [51]:
grp_df.columns.get_level_values(1)

Index(['amin', 'amax', 'mean', 'median', 'sum', 'mean'], dtype='object')

In [61]:
# Flattening #1
# to_flat_index, will push these values into a tuple
flat_df = grp_df.copy()
flat_df.columns = flat_df.columns.to_flat_index()
flat_df

,"(quality, amin)","(quality, amax)","(quality, mean)","(pH, median)","(pH, sum)","(pH, mean)"
wine,,,,,,
red,3,8,5.636023,3.31,5294.47,3.311113
white,3,9,5.877909,3.18,15616.13,3.188267


In [62]:
# Flattening #2
# We can join levels into a seperated
flat_df = grp_df.copy()
flat_df.columns = ['_'.join(col) for col in flat_df.columns.to_flat_index()]
flat_df

,quality_amin,quality_amax,quality_mean,pH_median,pH_sum,pH_mean
wine,,,,,,
red,3,8,5.636023,3.31,5294.47,3.311113
white,3,9,5.877909,3.18,15616.13,3.188267


In [ ]:
# Flattening #3
# We could substiture to_flat_index with the column values
flat_df = grp_df.copy()
flat_df.columns = ['_'.join(col) for col in grp_df.columns.values]
flat_df

# Long & Wide Formats

## Melt
* Converts to Long Format
* Think unpivot, converting a matrix into rows of cell values

In [73]:
# Create a simple frame
example_df = pd.DataFrame({'Name': ['A', 'B', 'C'],
                           'ColA': [1, 2, 3],
                           'ColB': ['a', 'b', 'c']})
example_df

,Name,ColA,ColB
0,A,1,a
1,B,2,b
2,C,3,c


In [74]:
# Default Melt will collapse your frame into variable -> value pairs
example_df.melt()

,variable,value
0,Name,A
1,Name,B
2,Name,C
3,ColA,1
4,ColA,2
5,ColA,3
6,ColB,a
7,ColB,b
8,ColB,c


In [75]:
# Extension 1 - id_vars
# These act as a grouping for each variable -> value pair
example_df.melt(id_vars=['Name'])

,Name,variable,value
0,A,ColA,1
1,B,ColA,2
2,C,ColA,3
3,A,ColB,a
4,B,ColB,b
5,C,ColB,c


In [77]:
# Extension 2 - value_vars
# These act as a filter, which variable -> value pairs should we generate
# Technically, these are the columns to unpivot
example_df.melt(value_vars=['Name'])

,variable,value
0,Name,A
1,Name,B
2,Name,C


In [78]:
# Extension 3 - var_name, value_name
# Customize the column names returned from melt
example_df.melt(value_vars=['Name'], var_name='Custom Variable', value_name='Custom Value')

,Custom Variable,Custom Value
0,Name,A
1,Name,B
2,Name,C


## Stack / Unstack

In [116]:
# Stack the columns as rows within each index
stacked_df = example_df.stack()
stacked_df

0  Name    A
   ColA    1
   ColB    a
1  Name    B
   ColA    2
   ColB    b
2  Name    C
   ColA    3
   ColB    c
dtype: object

In [118]:
# Reverse the stack operation
stacked_df.unstack()

,Name,ColA,ColB
0,A,1,a
1,B,2,b
2,C,3,c


# Crosstab

In [90]:
# Let's binarize quality into low / high
binned_df = wine_df.copy()
binned_df['quality_label'] = np.where(binned_df.quality < 6, 'Low', 'High')

In [93]:
# What are the counts of different combinations of wine and quality
pd.crosstab(binned_df.wine, binned_df.quality_label)

quality_label,High,Low
wine,,
red,855,744
white,3258,1640


In [106]:
# What is the mean pH value of wine / quality combinations
pd.crosstab(binned_df.wine, binned_df.quality_label, values=binned_df.pH, aggfunc='mean')

quality_label,High,Low
wine,,
red,3.310643,3.311653
white,3.197231,3.170457


# Pivot & Pivot Table

## Pivot

In [80]:
# Create a simple frame
example_df = pd.DataFrame({'Name': ['A', 'B', 'C'],
                           'ColA': [1, 2, 3],
                           'ColB': ['', 'b', 'c']})
example_df

,Name,ColA,ColB
0,A,1,a
1,B,2,b
2,C,3,c


In [99]:
# Define the index, columns, and fields to add to 
example_df.pivot(index='Name', columns='ColA', values='ColB')

ColA,1,2,3
Name,,,
A,a,NaN,NaN
B,NaN,b,NaN
C,NaN,NaN,c


## Pivot Table
* Pivot lacks the ability to aggregate
* Meaning, you have to have unique combinations to pivot

In [101]:
# Let's binarize quality into low / high
binned_df = wine_df.copy()
binned_df['quality_label'] = np.where(binned_df.quality < 6, 'Low', 'High')

In [102]:
# Similar to crosstab
pd.pivot_table(binned_df, index='wine', columns='quality_label', values='alcohol', aggfunc='mean')

quality_label,High,Low
wine,,
red,10.855029,9.926478
white,10.848880,9.849530


In [103]:
# But more powerful - can pass multiple aggfunc
pd.pivot_table(binned_df, index='wine', columns='quality_label', values='alcohol', aggfunc=['min', 'mean', 'max'])

min            mean             max      
quality_label High  Low       High       Low  High   Low
wine                                                    
red            8.4  8.4  10.855029  9.926478  14.0  14.9
white          8.5  8.0  10.848880  9.849530  14.2  13.6

In [107]:
# And allows for multiple values to be aggregates
pd.pivot_table(binned_df, index='wine', columns='quality_label', values=['alcohol', 'pH'], aggfunc=['min', 'mean', 'max'])

min                   ...     max                  
              alcohol         pH        ... alcohol          pH      
quality_label    High  Low  High   Low  ...    High   Low  High   Low
wine                                    ...                          
red               8.4  8.4  2.86  2.74  ...    14.0  14.9  4.01  3.90
white             8.5  8.0  2.72  2.79  ...    14.2  13.6  3.82  3.79

[2 rows x 12 columns]

In [108]:
# As well as stacking index or columns
pd.pivot_table(binned_df, index=['wine', 'quality_label'], values=['alcohol', 'pH'], aggfunc=['min', 'mean', 'max'])

min             mean               max      
                    alcohol    pH    alcohol        pH alcohol    pH
wine  quality_label                                                 
red   High              8.4  2.86  10.855029  3.310643    14.0  4.01
      Low               8.4  2.74   9.926478  3.311653    14.9  3.90
white High              8.5  2.72  10.848880  3.197231    14.2  3.82
      Low               8.0  2.79   9.849530  3.170457    13.6  3.79

In [120]:
# And...Can add margins
pd.pivot_table(binned_df, index=['wine'], values=['alcohol', 'pH'], aggfunc='mean', margins=True)

,alcohol,pH
wine,,
red,10.422983,3.311113
white,10.514267,3.188267
All,10.491801,3.218501


# Extras
* Corgies & Cats
* https://pandas.pydata.org/pandas-docs/version/0.24.0/user_guide/reshaping.html